In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

# If using group_unknown, the output is: X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid
# Otherwise: X_train, Y_train, X_valid, Y_valid

/usr/local/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train

In [3]:
# Helper function
def get_shape_after_skipping(initial_shape, strides_list):
    h, w = initial_shape
    for strides in strides_list:
        h = math.ceil(h / strides[0])
        w = math.ceil(w / strides[1])
    return h, w

In [4]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm_random_input_modify_v2 import *

In [5]:
graph_name = 'SpectrogramMultiLSTMRandomInputModify2'  # Specify graph class name

g_cnfg = Config('graph_02_lower_lr_lr_decay')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[-1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

g_cnfg.lstm_state_sizes = [1024, 1024]
g_cnfg.lstm_keep_prob = 0.75

g_cnfg.flat_hiddens = [512, 512, 50, 50]

#g_cnfg.conv1_keep_prob = 0.9
#g_cnfg.conv1_kernel_size = (10, 10)  # height, width
#g_cnfg.conv1_n_filters = 128
#g_cnfg.conv1_mp_size = (2, 1)
#g_cnfg.conv1_mp_strides = (2, 1)

#g_cnfg.conv2ds_keep_prob = 0.9
#g_cnfg.conv2ds_kernel_size = (3, 3)
#g_cnfg.conv2ds_strides = (1, 1)
#g_cnfg.conv2ds_n_filters = 128

#strides_list = [g_cnfg.conv1_mp_strides, g_cnfg.conv2ds_strides]
#g_cnfg.n_per_time_after_conv = get_shape_after_skipping((g_cnfg.X_img_h, g_cnfg.X_img_w), strides_list)[0] * g_cnfg.conv2ds_n_filters
#g_cnfg.n_per_time_after_reducing_conv = 128

g_cnfg.lr_initial = 0.01
g_cnfg.lr_decay_steps = 50000
g_cnfg.lr_decay_rate = 0.1  # 1.0 means no decay

t_cnfg = Config('run_01_group_unknown')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.random_pick_unknown = True
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label(['_background_noise_'])])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = int(int(XY_train_valid[0].shape[0] / 11 * 12) / t_cnfg.batch_size / 2)
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [6]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_02_lower_lr_lr_decay
X_img_h: 128
X_img_w: 71
Y_vector_len: 12
flat_hiddens: [512, 512, 50, 50]
lr_decay_rate: 0.1
lr_decay_steps: 50,000
lr_initial: 0.01
lstm_keep_prob: 0.75
lstm_state_sizes: [1024, 1024]
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8

name: run_01_group_unknown
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 79
max_step: 100,000,000
n_ave_ll_valid: 8
random_pick_unknown: 1
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown
Epoch size is 23,815 | Batch size is 150 | 158 steps per epoch
115 leftover gets discarded at the end of every epoch

Training starts @ 01/11/2018 05:45:32
Epoch 1 Step 1 Best model saved @ 01/11/2018 05:45:37 [Ave valid] 2.485
Epoch 1 Step 1 ends @ 01/11/

Epoch 30 Step 4,740 Best model saved @ 01/11/2018 06:26:53 [Ave valid] 0.672
Epoch 30 Step 4,740 ends @ 01/11/2018 06:26:53 [Train] 0.641, 78.7% [Valid] 79.1% [Ave valid] 0.672
Epoch 31 Step 4,819 Best model saved @ 01/11/2018 06:27:35 [Ave valid] 0.667
Epoch 31 Step 4,898 Best model saved @ 01/11/2018 06:28:16 [Ave valid] 0.655
Epoch 31 Step 4,898 ends @ 01/11/2018 06:28:16 [Train] 0.458, 87.3% [Valid] 82.4% [Ave valid] 0.655
Epoch 32 Step 4,977 Best model saved @ 01/11/2018 06:28:58 [Ave valid] 0.629
Epoch 32 Step 5,056 Best model saved @ 01/11/2018 06:29:39 [Ave valid] 0.621
Epoch 32 Step 5,056 ends @ 01/11/2018 06:29:39 [Train] 0.495, 82.7% [Valid] 81.8% [Ave valid] 0.621
Epoch 33 Step 5,135 Best model saved @ 01/11/2018 06:30:20 [Ave valid] 0.616
Epoch 33 Step 5,214 Best model saved @ 01/11/2018 06:31:01 [Ave valid] 0.599
Epoch 33 Step 5,214 ends @ 01/11/2018 06:31:01 [Train] 0.417, 86.7% [Valid] 82.9% [Ave valid] 0.599
Epoch 34 Step 5,293 Best model saved @ 01/11/2018 06:31:43 [A

Epoch 71 Step 11,218 ends @ 01/11/2018 07:23:17 [Train] 0.329, 91.3% [Valid] 86.7% [Ave valid] 0.362
Epoch 72 Step 11,376 ends @ 01/11/2018 07:24:39 [Train] 0.300, 92.7% [Valid] 90.4% [Ave valid] 0.360
Epoch 73 Step 11,534 ends @ 01/11/2018 07:26:01 [Train] 0.157, 94.7% [Valid] 91.1% [Ave valid] 0.348
Epoch 74 Step 11,692 ends @ 01/11/2018 07:27:24 [Train] 0.211, 92.7% [Valid] 90.3% [Ave valid] 0.350
Epoch 75 Step 11,850 Best model saved @ 01/11/2018 07:28:46 [Ave valid] 0.327
Epoch 75 Step 11,850 ends @ 01/11/2018 07:28:46 [Train] 0.375, 89.3% [Valid] 91.1% [Ave valid] 0.327
Epoch 76 Step 11,929 Best model saved @ 01/11/2018 07:29:28 [Ave valid] 0.325
Epoch 76 Step 12,008 ends @ 01/11/2018 07:30:09 [Train] 0.449, 85.3% [Valid] 88.2% [Ave valid] 0.334
Epoch 77 Step 12,166 ends @ 01/11/2018 07:31:32 [Train] 0.271, 90.0% [Valid] 91.0% [Ave valid] 0.329
Epoch 78 Step 12,324 Best model saved @ 01/11/2018 07:32:55 [Ave valid] 0.323
Epoch 78 Step 12,324 ends @ 01/11/2018 07:32:55 [Train] 0.1

Epoch 128 Step 20,224 Best model saved @ 01/11/2018 08:41:43 [Ave valid] 0.232
Epoch 128 Step 20,224 ends @ 01/11/2018 08:41:43 [Train] 0.079, 97.3% [Valid] 93.4% [Ave valid] 0.232
Epoch 129 Step 20,382 Best model saved @ 01/11/2018 08:43:06 [Ave valid] 0.229
Epoch 129 Step 20,382 ends @ 01/11/2018 08:43:06 [Train] 0.107, 97.3% [Valid] 93.2% [Ave valid] 0.229
Epoch 130 Step 20,540 Best model saved @ 01/11/2018 08:44:28 [Ave valid] 0.229
Epoch 130 Step 20,540 ends @ 01/11/2018 08:44:28 [Train] 0.167, 94.7% [Valid] 92.7% [Ave valid] 0.229
Epoch 131 Step 20,698 ends @ 01/11/2018 08:45:51 [Train] 0.180, 94.7% [Valid] 93.3% [Ave valid] 0.230
INFO:tensorflow:checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown/hourly/model-20856 is not in all_model_checkpoint_paths. Manually adding it.
Epoch 132 Step 20,856 Hourly model saved @ 01/11/2018 08:47:13
Epoch 132 Step 20,856 ends @ 01/11/2018 08:47:13 [Train] 0.177, 94.7% [Valid] 92.4% [Ave valid] 0.23

Epoch 186 Step 29,388 ends @ 01/11/2018 10:01:36 [Train] 0.079, 97.3% [Valid] 93.9% [Ave valid] 0.205
Epoch 187 Step 29,546 ends @ 01/11/2018 10:02:59 [Train] 0.089, 97.3% [Valid] 94.1% [Ave valid] 0.206
Epoch 188 Step 29,704 ends @ 01/11/2018 10:04:21 [Train] 0.046, 98.7% [Valid] 94.7% [Ave valid] 0.202
Epoch 189 Step 29,783 Best model saved @ 01/11/2018 10:05:03 [Ave valid] 0.199
Epoch 189 Step 29,862 ends @ 01/11/2018 10:05:43 [Train] 0.165, 95.3% [Valid] 93.7% [Ave valid] 0.199
Epoch 190 Step 30,020 ends @ 01/11/2018 10:07:06 [Train] 0.204, 95.3% [Valid] 94.1% [Ave valid] 0.208
Epoch 191 Step 30,178 ends @ 01/11/2018 10:08:28 [Train] 0.088, 98.7% [Valid] 94.0% [Ave valid] 0.208
Epoch 192 Step 30,336 ends @ 01/11/2018 10:09:50 [Train] 0.035, 99.3% [Valid] 94.7% [Ave valid] 0.207
Epoch 193 Step 30,494 ends @ 01/11/2018 10:11:12 [Train] 0.094, 96.7% [Valid] 94.2% [Ave valid] 0.208
Epoch 194 Step 30,652 ends @ 01/11/2018 10:12:35 [Train] 0.130, 95.3% [Valid] 94.0% [Ave valid] 0.201
Epo

Epoch 255 Step 40,211 Best model saved @ 01/11/2018 11:35:39 [Ave valid] 0.188
Epoch 255 Step 40,290 ends @ 01/11/2018 11:36:20 [Train] 0.101, 97.3% [Valid] 94.7% [Ave valid] 0.190
Epoch 256 Step 40,448 ends @ 01/11/2018 11:37:42 [Train] 0.077, 98.0% [Valid] 95.0% [Ave valid] 0.189
Epoch 257 Step 40,606 ends @ 01/11/2018 11:39:05 [Train] 0.149, 96.7% [Valid] 94.3% [Ave valid] 0.194
Epoch 258 Step 40,764 ends @ 01/11/2018 11:40:27 [Train] 0.091, 98.7% [Valid] 94.7% [Ave valid] 0.198
Epoch 259 Step 40,922 ends @ 01/11/2018 11:41:50 [Train] 0.067, 98.0% [Valid] 94.6% [Ave valid] 0.201
Epoch 260 Step 41,080 ends @ 01/11/2018 11:43:12 [Train] 0.069, 98.7% [Valid] 94.7% [Ave valid] 0.209
Epoch 261 Step 41,238 ends @ 01/11/2018 11:44:34 [Train] 0.151, 96.7% [Valid] 94.7% [Ave valid] 0.211
Epoch 262 Step 41,396 ends @ 01/11/2018 11:45:56 [Train] 0.102, 97.3% [Valid] 94.2% [Ave valid] 0.212
Epoch 263 Step 41,554 ends @ 01/11/2018 11:47:19 [Train] 0.132, 97.3% [Valid] 93.9% [Ave valid] 0.232
INF

Epoch 325 Step 51,350 ends @ 01/11/2018 13:12:27 [Train] 0.157, 96.0% [Valid] 95.5% [Ave valid] 0.175
Epoch 326 Step 51,508 ends @ 01/11/2018 13:13:50 [Train] 0.087, 98.0% [Valid] 95.4% [Ave valid] 0.175
Epoch 327 Step 51,666 ends @ 01/11/2018 13:15:12 [Train] 0.057, 99.3% [Valid] 95.6% [Ave valid] 0.178
Epoch 328 Step 51,824 ends @ 01/11/2018 13:16:35 [Train] 0.064, 98.7% [Valid] 95.4% [Ave valid] 0.179
Epoch 329 Step 51,982 ends @ 01/11/2018 13:17:56 [Train] 0.072, 98.7% [Valid] 95.6% [Ave valid] 0.179
Epoch 330 Step 52,140 ends @ 01/11/2018 13:19:19 [Train] 0.072, 97.3% [Valid] 95.5% [Ave valid] 0.178
Epoch 331 Step 52,298 ends @ 01/11/2018 13:20:41 [Train] 0.040, 99.3% [Valid] 95.4% [Ave valid] 0.176
Epoch 332 Step 52,456 ends @ 01/11/2018 13:22:04 [Train] 0.099, 98.0% [Valid] 95.4% [Ave valid] 0.176
Epoch 333 Step 52,614 ends @ 01/11/2018 13:23:26 [Train] 0.073, 97.3% [Valid] 95.5% [Ave valid] 0.173
Epoch 334 Step 52,693 Best model saved @ 01/11/2018 13:24:08 [Ave valid] 0.173
Epo

Epoch 397 Step 62,726 ends @ 01/11/2018 14:51:19 [Train] 0.104, 98.7% [Valid] 95.4% [Ave valid] 0.203
Epoch 398 Step 62,884 ends @ 01/11/2018 14:52:41 [Train] 0.098, 98.0% [Valid] 95.5% [Ave valid] 0.204
Epoch 399 Step 63,042 ends @ 01/11/2018 14:54:04 [Train] 0.070, 98.0% [Valid] 95.6% [Ave valid] 0.205
Epoch 400 Step 63,200 ends @ 01/11/2018 14:55:26 [Train] 0.063, 98.7% [Valid] 95.4% [Ave valid] 0.204
Epoch 401 Step 63,358 ends @ 01/11/2018 14:56:49 [Train] 0.088, 98.0% [Valid] 95.7% [Ave valid] 0.201
Epoch 402 Step 63,516 ends @ 01/11/2018 14:58:11 [Train] 0.075, 97.3% [Valid] 95.2% [Ave valid] 0.203
Epoch 403 Step 63,674 ends @ 01/11/2018 14:59:33 [Train] 0.041, 99.3% [Valid] 94.1% [Ave valid] 0.214
Epoch 404 Step 63,832 ends @ 01/11/2018 15:00:56 [Train] 0.123, 97.3% [Valid] 93.3% [Ave valid] 0.227
Epoch 405 Step 63,990 ends @ 01/11/2018 15:02:18 [Train] 0.023, 99.3% [Valid] 93.2% [Ave valid] 0.238
Epoch 406 Step 64,148 ends @ 01/11/2018 15:03:41 [Train] 0.128, 96.0% [Valid] 94.6

Epoch 475 Step 75,050 ends @ 01/11/2018 16:38:22 [Train] 0.091, 98.0% [Valid] 95.1% [Ave valid] 0.205
Epoch 476 Step 75,208 ends @ 01/11/2018 16:39:44 [Train] 0.098, 98.7% [Valid] 95.6% [Ave valid] 0.208
Epoch 477 Step 75,366 ends @ 01/11/2018 16:41:06 [Train] 0.084, 97.3% [Valid] 94.9% [Ave valid] 0.210
Epoch 478 Step 75,524 ends @ 01/11/2018 16:42:29 [Train] 0.068, 99.3% [Valid] 94.5% [Ave valid] 0.216
Epoch 479 Step 75,682 ends @ 01/11/2018 16:43:51 [Train] 0.052, 98.7% [Valid] 95.5% [Ave valid] 0.214
Epoch 480 Step 75,840 ends @ 01/11/2018 16:45:13 [Train] 0.173, 97.3% [Valid] 95.2% [Ave valid] 0.213
Epoch 481 Step 75,998 ends @ 01/11/2018 16:46:35 [Train] 0.086, 97.3% [Valid] 95.4% [Ave valid] 0.215
Epoch 482 Step 76,156 ends @ 01/11/2018 16:47:58 [Train] 0.049, 98.7% [Valid] 95.7% [Ave valid] 0.209
Epoch 483 Step 76,314 ends @ 01/11/2018 16:49:20 [Train] 0.077, 98.0% [Valid] 95.7% [Ave valid] 0.208
INFO:tensorflow:checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_

KeyboardInterrupt: 